Author: Roel Faber

In [3]:
%load_ext autoreload
%autoreload 2

In [19]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

import getgames

In [24]:
from getgames import test, start_browser, create_df

Start automatic browser and go to website with football statistics

In [65]:
browser = getgames.start_browser()

In [68]:
columns = ["Season","Round","Home","Away","HomeGoals","AwayGoals","Result","HomePoints","AwayPoints"]
roundnrs = range(1,3)
startjaar = 2020
eindjaar = 2020

df = getgames.create_df(browser,columns,roundnrs,startjaar,eindjaar)

In [69]:
df

,Season,Round,Home,Away,HomeGoals,AwayGoals,Result,HomePoints,AwayPoints
0,2020-2021,1,sc Heerenveen,Willem II,2,0,1,3,0
1,2020-2021,1,PEC Zwolle,Feyenoord,0,2,2,0,3
2,2020-2021,1,FC Twente,Fortuna Sittard,2,0,1,3,0
3,2020-2021,1,FC Emmen,VVV-Venlo,3,5,2,0,3
4,2020-2021,1,Heracles Almelo,ADO Den Haag,2,0,1,3,0
5,2020-2021,1,Sparta Rotterdam,AFC Ajax,0,1,2,0,3
6,2020-2021,1,FC Groningen,PSV,1,3,2,0,3
7,2020-2021,1,RKC Waalwijk,Vitesse,0,1,2,0,3
8,2020-2021,2,VVV-Venlo,FC Utrecht,1,1,3,1,1
9,2020-2021,2,AZ Alkmaar,PEC Zwolle,1,1,3,1,1


In [ ]:
# df.to_csv('Data/matches.csv')
# df.to_pickle('Data/matches.pkl')

df.to_csv('Data/matches_20192020.csv')
df.to_pickle('Data/matches_20192020.pkl')